In [1]:
import pandas as pd
import xarray as xr
import numpy as np
from linopy import Model

# Technologies data:
tech_param = pd.read_csv('Data/technologies.csv',index_col=0)
# Storages data
sto_param = pd.read_csv("Data/storage_technologies.csv", index_col=0).to_xarray()
sto_param_cold = pd.read_csv("Data/storage_cold_technologies.csv", index_col=0).to_xarray()
# Demand:
demand = pd.read_csv('Data/demand_hourly_Abu_Dhabi.csv',index_col=0, delimiter = ';').to_xarray()
# COP
cop = pd.read_csv('Data/COP.csv',index_col=0, delimiter= ',')
# Global parameters:
general_parameters = pd.read_csv('Data/global_parameters.csv',index_col=0, delimiter= ',')
# Airchiller data:
conv_tech_param = pd.read_csv('Data/conversion_technologies.csv', index_col =0, delimiter = ',').to_xarray()

# Renewable generation:
AF = pd.read_csv('Data/AF.csv',index_col=0, delimiter = ';') # AF is given in the file only for the renewables, we need to add columns for the other technologies.
for tech in tech_param.index:
    if tech not in AF.columns:
        AF.loc[:,tech] = tech_param.loc[tech, "Availability"] 
tech_param = tech_param.to_xarray()

In [2]:
### Sets
Time = demand.get_index("Time")
Tech = tech_param.get_index("Technologies")
Locations = pd.Index(["Yas", "Mariah", "Muroor", "Reem"], name="Locations")  # creation of my set of each location

# Technologies related
TECH_MAX_CAP = tech_param['Maximum capacity']

MARGINAL_FUEL_COST = tech_param['Fuel costs']/tech_param['Rated efficiency']
CO2_INTENSITY = tech_param['Fuel CO2 content']/tech_param['Rated efficiency']
MARGINAL_CO2_EMISSION = tech_param['Fuel CO2 content']
TECH_PHI = tech_param["Discount rate"]/(1-(1+tech_param["Discount rate"])**(-tech_param["Lifetime"]))
TECH_IC = TECH_PHI*tech_param['Investment cost']
DELTA_MAX_UP = tech_param['Ramp-up rate']
DELTA_MAX_DOWN = tech_param['Ramp-down rate']
AF = AF[Tech]
AF  = xr.DataArray(AF,coords=[Time,Tech])
LEGACY = tech_param["Legacy capacity"]

# Electrical Storage related
STO_MAX_CAP = sto_param["Maximum capacity"]
STO_PHI = sto_param["Discount rate"]/(1-(1+sto_param["Discount rate"])**(-sto_param["Lifetime"]))
STO_IC = STO_PHI*sto_param["Investment cost"]
EFF_D = sto_param["Discharge efficiency"]
EFF_C = sto_param["Charge efficiency"]
SOC_MAX = sto_param["Maximum state of charge"]
SOC_MIN = sto_param["Minimum state of charge"]
STO_dT = sto_param["Storage duration"]

# Cold Storage related
STO_MAX_CAP_COLD = sto_param_cold["Maximum capacity"]/4
STO_PHI_COLD = sto_param_cold["Discount rate"]/(1-(1+sto_param_cold["Discount rate"])**(-sto_param_cold["Lifetime"]))
STO_IC_COLD = STO_PHI_COLD*sto_param_cold["Investment cost"]
EFF_D_COLD = sto_param_cold["Discharge efficiency"]
EFF_C_COLD = sto_param_cold["Charge efficiency"]
SOC_MAX_COLD = sto_param_cold["Maximum state of charge"]
SOC_MIN_COLD = sto_param_cold["Minimum state of charge"]
STO_dT_COLD = sto_param_cold["Storage duration"]


# Links Related
conv_tech_phi = conv_tech_param['Discount rate']/(1-(1+conv_tech_param['Discount rate'])**(-conv_tech_param['Lifetime']))
CONV_TECH_IC = conv_tech_phi*conv_tech_param['Investment cost']
CONV_TECH_MAX_CAP = conv_tech_param['Maximum capacity']

# demande
COP = cop["COP"]
COP_avg = COP.mean().item()                            # Use for the objective function (= 4)
COP  = xr.DataArray(COP,coords=[Time])
DEMAND_elec = demand["Campus_elec"]                    # Elec demand total in my network [MW_froid]
DEMAND_cold_total = demand["Campus_cold"]              # Cold demand total in my network [MW_froid]                 
DEMAND_cold = xr.Dataset( {"Yas": demand["Yas"],       # [MW_froid] Fixed cold demand of each VPP (= sum of the cold demande of each building in 4 locations)
                       "Mariah": demand["Mariah"],
                       "Muroor": demand["Muroor"],
                       "Reem": demand["Reem"]} ).to_array(dim="Locations")   
ELEC_DEMAND_cold_total = DEMAND_cold_total / COP       # total of the Electricity demand of cold in my network [MW_elec]

# Configuration and scenario parameters
TS = float(general_parameters.Value.loc['Time step'])
VOLL = float(general_parameters.Value.loc['VOLL'])
CO2_PRICE = float(general_parameters.Value.loc['CO2_price'])
CO2_BOUND = float(general_parameters.Value.loc['CO2_bound'])

In [3]:
### Create the model instance

m = Model()

### Create the variables

cap_tech = m.add_variables(lower = 0, coords = [Tech], name = 'cap_tech')          # Installed capacity of the technologies [MW]
g = m.add_variables(lower = 0, coords = [Tech, Time], name = 'g')                  # Generated power [MW]
fcd = m.add_variables(lower = 0, coords = [Tech, Time], name = 'fcd')              # Fuel cost definition [EUR/h]
ccd = m.add_variables(lower = 0, coords= [Tech, Time], name = 'ccd')               # CO2 cost definition [EUR/h]
cap_sto = m.add_variables(lower = 0, name = 'cap_sto')                             # Installed capacity of the storages [MWh]
sd = m.add_variables(lower = 0, coords = [Time], name = 'sd')                      # Storage discharge [MW]
sc = m.add_variables(lower = 0, coords = [Time], name = 'sc')                      # Storage charge [MW]
se = m.add_variables(lower = 0, coords = [Time], name = 'se')                      # Stored energy [MWh]

co2_emissions =  m.add_variables(lower = 0, name = 'co2_emissions')                # Combined CO2 emissions [tCO2eq]
inj = m.add_variables(lower = 0, coords=[Locations, Time], name = 'inj')           # Electricity required to meet the cold demand through the conversion system [MW]
ens_elec = m.add_variables(lower = 0, coords = [Time], name = 'ens')               # Energy no served for the electric part [MWh]
ens_cold = m.add_variables(lower = 0, coords=[Locations, Time], name = 'ens_cold') # Energy no served for the cold part [MWh]

#NO curtailment considered here

#Cold battery variables
vpp_c = m.add_variables(lower = 0, coords=[Locations, Time], name = 'vpp_c')       # Storage charge (Ice formation) [MW_cold]
vpp_e = m.add_variables(lower = 0, coords=[Locations, Time], name = 'vpp_e')       # Stored energy [MWh_cold]

# optimize the size of the storage
e_max = m.add_variables(lower = 0, coords=[Locations], name = 'e_max')             # Installed capacity of the storages [MWh_cold] 
c_max = m.add_variables(lower = 0, coords=[Locations], name = "c_max")             # Installed capacity of the storages [MWh]


### Create the constraints

# Technology sizing
cap_max_tech = m.add_constraints(cap_tech <= TECH_MAX_CAP, name = "cap_max_tech")
power_max = m.add_constraints(g <= AF*cap_tech, name= 'power_max')

# Ramping up/down constraints (limit the variation of the power production)
power_rampup_initial = m.add_constraints(g.loc[:,:Time[0]] - g.loc[:,Time[len(Time)-1]:] <= DELTA_MAX_UP*cap_tech , name = 'power_rampup_initial')
power_rampup = m.add_constraints(g.loc[:, Time[1:]] <= g.shift(Time = 1) + DELTA_MAX_UP*cap_tech, name = 'power_rampup')

power_rampdown_initial = m.add_constraints(g.loc[:,:Time[0]] - g.loc[:,Time[len(Time)-1]:]  >= - DELTA_MAX_DOWN*cap_tech, name = 'power_rampdown_initial')
power_rampdown = m.add_constraints(g.loc[:, Time[1:]]>= g.shift(Time = 1) - DELTA_MAX_DOWN*cap_tech, name = 'power_rampdown')


# Electrical Storage sizing + Initial condition
cap_max_sto = m.add_constraints(cap_sto <= STO_MAX_CAP, name = "cap_max_sto")
storage_balance = m.add_constraints(se.loc[Time[1:]] - se.shift(Time = 1) == TS*EFF_C*sc.loc[Time[1:]] -TS*sd.loc[Time[1:]]/EFF_D, name = 'storage_balance')
storage_charge_max = m.add_constraints(sc <= cap_sto/STO_dT, name = 'storage_charge_max')
storage_in_fin = m.add_constraints(se.loc[:Time[0]] == se.loc[Time[len(Time)-1]:] , name = 'storage_in_fin')

storage_discharge_max = m.add_constraints(sd <= cap_sto/STO_dT, name = 'storage_discharge_max')
storage_energy_max = m.add_constraints(se <= SOC_MAX*cap_sto, name = 'storage_energy_max' )
storage_energy_min = m.add_constraints(se >= SOC_MIN*cap_sto, name = 'storage_energy_min' )


# Cold storage sizing
for loc in Locations:
    m.add_constraints(e_max.loc[loc] <= STO_MAX_CAP_COLD, name =f"cap_max_sto_cold_{loc}")
    m.add_constraints(vpp_e.loc[loc, Time[1:]] == vpp_e.loc[loc, Time[:-1]] + TS*(EFF_C_COLD * vpp_c.loc[loc, Time[:-1]] - DEMAND_cold.loc[loc, Time[:-1]]/EFF_D_COLD), name=f"storage_balance_cold_{loc}")
    m.add_constraints(vpp_e.loc[loc, Time[0]] == vpp_e.loc[loc, Time[-1]] + TS*(EFF_C_COLD * vpp_c.loc[loc, Time[-1]] - DEMAND_cold.loc[loc, Time[-1]]/EFF_D_COLD), name=f"storage_cyclic_cold_{loc}")  

    m.add_constraints(vpp_c.loc[loc] <= e_max.loc[loc]*STO_dT_COLD, name =f"vpp_max_c_{loc}")   
    m.add_constraints(vpp_e.loc[loc] <= SOC_MAX_COLD*e_max.loc[loc], name =f"vpp_max_e_{loc}")
    m.add_constraints(vpp_e.loc[loc] >= SOC_MIN_COLD*e_max.loc[loc], name =f"vpp_min_e_{loc}")

# Market clearing
mcc = m.add_constraints(g.sum(dims = 'Technologies') + sd - sc == - ens_elec + DEMAND_elec + inj.sum(dims='Locations') , name = 'mcc')
for loc in Locations:
    m.add_constraints(inj.loc[loc]*COP + ens_cold.loc[loc] == vpp_c.loc[loc], name =f"mcc2_{loc}")

# Limit on energy not served
ens_lim = m.add_constraints(ens_elec<=DEMAND_elec, name = 'ens_lim')
for loc in Locations:
    m.add_constraints(ens_cold.loc[loc] <= DEMAND_cold.loc[loc], name =f"ens_lim2_{loc}")
    m.add_constraints(inj.loc[loc]*COP <= c_max.loc[loc], name=f"max_flow_conv_tech_{loc}")

# Costs
fuel_cost = m.add_constraints(fcd == MARGINAL_FUEL_COST*g, name="fuel_cost")

# CO2 Emissions
co2_account =  m.add_constraints(co2_emissions == (CO2_INTENSITY*g).sum(dims=["Technologies", "Time"]), name="co2 account")
co2 = m.add_constraints(ccd == CO2_PRICE*MARGINAL_CO2_EMISSION*g, name = 'co2')


# Add the objective function
total_cost = (TECH_IC*cap_tech).sum(dim="Technologies") + (c_max.sum(dim='Locations')*float(CONV_TECH_IC)) + (cap_sto*STO_IC) + (e_max.sum(dim='Locations')*float(STO_IC_COLD)) + (TS*fcd + TS*ccd).sum(dims = ['Technologies', 'Time']) + (VOLL*TS*ens_elec).sum(dims ='Time') + (ens_cold.sum(dims='Locations')*float(VOLL*TS)).sum(dims ='Time')
m.add_objective(total_cost)

In [4]:
m.solve("gurobi", BarHomogeneous = True, BarConvTol=1e-4, MIPGap=0.0002)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2551043


GurobiError: SSL: no alternative certificate subject name matches target hostname 'token.gurobi.com' (code 60, command POST https://token.gurobi.com/api/v1/tokens)

## Post-process

In [ ]:
m.objective

In [ ]:
import hvplot.xarray
import hvplot.pandas
import panel as pn
import panel.widgets as pnw
import matplotlib.pyplot as plt

In [ ]:
# Vérifications pour le modèle décentralisé:
print("État maximum de la batterie :", se.solution.max(dim='Time').item(), "MWh") 
print("État maximum du stockage de froid :", vpp_e.solution.sum(dim='Locations').max(dim='Time').item(), "MWh")
print("Valeur maximum de charge froide durant l’année :", vpp_c.solution.sum(dim='Locations').max(dim='Time').item(), "MW")
print("Demande totale de froid sur l’année :", DEMAND_cold_total.sum().item(), "MWh")

In [ ]:
for loc in Locations:
    print(f'For the location {loc}:')
    print(f'  - e_max = {e_max.solution.sel({"Locations": loc}).values}')
    print(f'  - c_max = {c_max.solution.sel({"Locations": loc}).values}')

In [ ]:
# --- Color palette (soft & pro) ---
lightblue = "#89CFF0"  
purple = "#bcbddc"
darkblue = "#003366"

# --- Data selection ---
loc = "Yas"

# --- Data series ---
cold_demand = DEMAND_cold.sel(Locations=loc, Time=sample_day)
storage = vpp_e.solution.sel(Locations=loc, Time=sample_day)
charge = vpp_c.solution.sel(Locations=loc, Time=sample_day)

# --- Plot creation ---
fig, ax1 = plt.subplots(figsize=(12, 6))

ln1 = ax1.plot(hours, cold_demand, marker='o', label="Cold demand (MW)", color=purple)
ln2 = ax1.plot(hours, storage, linestyle='--', marker='s', label="Storage state (MWh)", color=lightblue)
ln3 = ax1.plot(hours, charge, linestyle='-.', marker='^', label="Cold storage charging (MW)", color=darkblue)
ax1.set_xlabel("Hour")
ax1.set_ylabel("Cold demand, charging (MW) and storage level (MWh)")
ax1.set_xticks(hours)
ax1.grid(True)

lns = ln1 + ln2 + ln3
labels = [l.get_label() for l in lns]
ax1.legend(lns, labels, loc='upper right')

plt.title("Decentralized – Cold demand, storage and charging in one location for a representative day")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

for loc in vpp_e.solution.Locations.values:
    series = vpp_e.solution.sel(Locations=loc, Time=sample_day)
    plt.plot(hours, series.values, label=loc, color=colors[loc], linewidth=2)

plt.xlabel("Hour")
plt.ylabel("Stored energy (MWh)")
plt.title("Cold storage state by location for a representative day")
plt.legend(title="Location")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
lightblue = "#89CFF0"  
purple = "#bcbddc"
darkblue = "#003366"
red = "#c994c7"

# 0) Préparation
locations = vpp_c.solution.Locations.values
sample_day = vpp_c.solution.Time[:24] 
hours = np.arange(len(sample_day))

# 1) Création de la figure 2×2
fig, axes = plt.subplots(2, 2, figsize=(15, 10), sharex=True)
axes = axes.flatten()

for ax, loc in zip(axes, locations):
    ax2 = ax.twinx()
    
    # Séries “froid”
    cold_demand = DEMAND_cold.sel(Locations=loc, Time=sample_day)                
    storage     = vpp_e.solution.sel(Locations=loc, Time=sample_day)  
    charge      = vpp_c.solution.sel(Locations=loc, Time=sample_day)

    # Tracé des froids
    ax.plot(hours, storage, label="État stockage", marker='s', color = lightblue)
    ax.plot(hours, charge, label="Charge VPP", marker='o', color = purple)
    ax.set_ylabel("Froid (MW / MWh)")
    ax2.plot(hours, cold_demand, label="Demande froid", linestyle='--', marker='^', color = darkblue)
    ax2.set_ylabel("Demande froid (MW)")

    # Légendes combinées
    lines, labels = ax.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(lines + lines2, labels + labels2, loc="upper left")
    
    ax.set_title(f"Quartier {loc}")
    ax.tick_params(axis='x', rotation=45)

fig.suptitle("Comparaison demande de Froid vs Réponse VPP par Quartier", y=1.02, fontsize=16)
plt.tight_layout()
plt.show()


In [ ]:
# ---- Paramètres ----
sample_day = vpp_c.solution.Time[265:289]
hours = np.arange(1, len(sample_day) + 1)

# ---- Extraction ----
# Production par techno 
df_prod = g.solution.sel(Time=sample_day).to_pandas().T  
if 'Units' in df_prod.columns:  
    df_prod = df_prod.set_index('Units')

prod_cols = list(df_prod.columns)

# PV, Wind, GT, Grid (ou adapte selon tes colonnes)
prod_stack = [df_prod[c].values for c in prod_cols if c in ['PV', 'Wind', 'GT', 'Grid']]

# Demande élec pour le froid (inj, demande d'élec au chiller pour ce quartier)
inj_loc = inj.solution.sum(dim='Locations').sel(Time=sample_day).values

# Demande élec hors froid
dem_elec = DEMAND_elec.sel(Time=sample_day).values

# Charge/décharge batterie (élec)
charge_batt = sc.solution.sel(Time=sample_day).values
discharge_batt = sd.solution.sel(Time=sample_day).values
eta_batt = se.solution.sel(Time=sample_day).values

# Demande d'électricité totale pour le froid sur tout le campus
elec_cold_total = ELEC_DEMAND_cold_total.sel(Time=sample_day).values

plt.figure(figsize=(15,7))

# Stack de production
labels_prod = [c for c in prod_cols if c in ['PV', 'Wind', 'GT', 'Grid']]
plt.stackplot(hours, *prod_stack, labels=labels_prod, alpha=0.5)

# Non-cooling electric demand
plt.plot(hours, dem_elec, '--', color='steelblue', linewidth=2, label='Total electric demand')

# Total campus cooling electric demand
plt.plot(hours, elec_cold_total, ':', color='black', linewidth=2, label='Total cooling electric demand')

# Cooling electric demand (local chiller)
plt.plot(hours, inj_loc, 'o-', color='orangered', linewidth=2.5, label='Total cold storage charging')

# Battery actions
plt.plot(hours, charge_batt, 'v-', color='green', label='Battery charging', alpha=0.7)
plt.plot(hours, -discharge_batt, '^-', color='lime', label='Battery discharging', alpha=0.7)
plt.plot(hours, eta_batt, '^-', color='slateblue', label='Battery stat', alpha=0.7)

plt.xlabel('Hour')
plt.ylabel('Power [MW]')
plt.title('Energy profile of the entire campus')
plt.legend(loc='upper left', ncol=2)
plt.grid(True, linestyle=':')
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Assuming 'g' is already defined and contains the solution dataframe
generation_df = g.solution.to_dataframe().unstack(level=0)
generation_df.columns = generation_df.columns.droplevel(0)

# Calculate total production for each column
total_production = generation_df.sum()

# Sort columns by total production in descending order
sorted_columns = total_production.sort_values(ascending=False).index
# Sort values within each column in descending order
generation_df = generation_df[sorted_columns].apply(lambda x: x.sort_values(ascending=False).values, axis=0)

bottom_stack = np.zeros(8760)
demand = DEMAND_elec.to_series().sort_values(ascending=False).values
#plt.fill_between(range(8760), bottom_stack, demand, label='Battery Discharge', alpha=0.7)

# Ensure the DataFrame has 8760 rows
if generation_df.shape[0] != 8760:
    raise ValueError("The DataFrame does not have 8760 rows. Check the input data.")

# Initialize the bottom stack for the stacked area plot
bottom_stack = np.zeros(8760)

# Plot the load duration curve
for column in generation_df.columns:
    plt.fill_between(range(8760), bottom_stack, bottom_stack + generation_df[column], label=column, alpha=0.7)
    bottom_stack += generation_df[column]

# Convert DEMAND DataArray to a pandas Series and sort it in descending order
demand = DEMAND_elec.to_series().sort_values(ascending=False).values
#plt.plot(range(8760), demand, color='black', label='Demand', linewidth=2)


plt.xlabel("Time [h]")
plt.ylabel("Power Generation [MW]")
plt.title("Yearly power generation")
plt.legend()
plt.grid()
plt.show()

In [ ]:
cap_tech.solution.to_dataframe().plot(kind='bar')
print(cap_tech.solution.values)